<a href="https://colab.research.google.com/github/yinhang945/CUS635/blob/main/Practice_HW1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sh

     |████████████████████████████████| 40 kB 3.0 MB/s 


In [32]:
import pandas as pd
import ftplib
from sh import gunzip
from glob import glob
import xml.etree.ElementTree as ET
import random

In [3]:
ftp = ftplib.FTP('ftp.ncbi.nlm.nih.gov')
ftp.login()
ftp.cwd('/pubmed/baseline/')
files = ftp.nlst()
for file in files[:10]:
  if file[-3:] == '.gz': 
    print("Downloading..." + file)
    ftp.retrbinary("RETR " + file ,open('/content/' + file, 'wb').write)
    gunzip(file)
ftp.close()

Downloading...pubmed22n0003.xml.gz
Downloading...pubmed22n0002.xml.gz
Downloading...pubmed22n0004.xml.gz
Downloading...pubmed22n0001.xml.gz
Downloading...pubmed22n0005.xml.gz


In [26]:
data =  []                               # INITIALIZE DATA LIST
for xml_file in glob('*.xml'):
    for event, elem in ET.iterparse(xml_file, events=('start', 'end')):
        if event == 'start':
            if elem.tag == "PubmedArticle":
                pub = {}                 # INITIALIZE ARTICLE DICT

            if elem.tag == 'ArticleTitle':
                pub["Title"] = elem.text
                #pub["Abstract"] = []

            if elem.tag == 'AbstractText':
                pub["Abstract"] = elem.text
                

        if event == 'end':
            if elem.tag == "PubmedArticle":
                #pub["Source"] = xml_file
                data.append(pub)         # APPEND MULTIPLE ARTICLES

        elem.clear()

In [27]:
data[56]

{'Abstract': "The interpretation of the morphological features of alcoholic hepatitis is discussed in terms of a comparison with the results of an ultrastructural and histoenzymological study of the liver biopsies of nine patients. In these patients liver biopsies were performed in the initial stage of the illness and fifteen days after five were re-biopsied, when the clinical and biological signs were improved. The correlations between morphological and biological data were good, especially for the levels of serological and histoenzymological alkaline phosphatase and gamma-glutamyltranspeptidase evaluations. However, when histological appearances had returned to normal, after two weeks of abstinence from alcohol several histological and ultrastructural features of the initial hepatitis persisted. The presence of evolving cirrhosis was a contributing factor to the severity of the changes seen. Morphologically, apart from the changes due to chronic alcoholic intoxication (steatosis, mit

In [28]:
df = pd.DataFrame(data)
df['Concatenate'] = df.fillna('').sum(axis=1)
df.head()

,Title,Abstract,Concatenate
0,[Beta-blockers and arterial hypertension in th...,NaN,[Beta-blockers and arterial hypertension in th...
1,[beta-blockers and high risk pregnancies. View...,NaN,[beta-blockers and high risk pregnancies. View...
2,[Surface defense mechanisms of the nasal mucosa].,NaN,[Surface defense mechanisms of the nasal mucosa].
3,[Pharmacological treatment of coronary disease].,NaN,[Pharmacological treatment of coronary disease].
4,Treating the outpatient schizophrenic.,NaN,Treating the outpatient schizophrenic.


In [30]:
final_df = df['Concatenate']
final_df.tail()

149995    [Place of beta-blockers in the treatment of es...
149996    [Long-term utilization of beta-blockers in art...
149997    [beta-blockers in hypertension with renal fail...
149998    [Treatment by beta-blockers of arterial hypert...
149999    [Beta-blockers and arterial hypertension in th...
Name: Concatenate, dtype: object

In [31]:
final_df.to_csv('pubmed_abstracts.csv', encoding='utf-8')

In [33]:
def generate_sample(n):
  filename = 'pubmed_abstracts.csv'
  total = sum(1 for line in open(filename)) - 1 #number of records in file (excludes header)
  sample = n #desired sample size
  skip = sorted(random.sample(range(1,total+1),total-sample)) #the 0-indexed header will not be included in the skip list
  sample_df = pd.read_csv(filename, skiprows=skip)
  return sample_df

In [35]:
sample_df = generate_sample(100000)

In [36]:
sample_df

,Unnamed: 0,Concatenate
0,0,[Beta-blockers and arterial hypertension in th...
1,2,[Surface defense mechanisms of the nasal mucosa].
2,3,[Pharmacological treatment of coronary disease].
3,4,Treating the outpatient schizophrenic.
4,5,[An evaluation of lorazépam (temesta) in out-p...
...,...,...
99995,149994,[Utilization of pindolol in arterial hypertens...
99996,149995,[Place of beta-blockers in the treatment of es...
99997,149996,[Long-term utilization of beta-blockers in art...
99998,149997,[beta-blockers in hypertension with renal fail...


In [37]:
sample_df.to_csv('<The Avengers>_pubmed_sample.csv')